In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import numpy as np
import MySQLdb
from pyecharts import Line, Bar
# 求序列的
def MA(arr, ma=30):
    res = []
    for i in range(ma-1, len(arr)):
        res.append(np.mean(arr[i-ma+1:i+1]))
    return res
# 皮尔逊相关度
def corr(X, Y):
    sumX = 0
    sumY = 0
    sumXY = 0
    sumX2 = 0
    sumY2 = 0
    n = len(X)
    for i in xrange(n):
        sumX += X[i]
        sumY += Y[i]
        sumXY += X[i] * Y[i]
        sumX2 += X[i]**2
        sumY2 += Y[i]**2
    return (sumXY - sumX*sumY/n) / math.sqrt((sumX2-sumX**2/n)*(sumY2-sumY**2/n))

In [2]:
conn = MySQLdb.connect(host='localhost', user='root', passwd='codegeass', db='future_data', charset='utf8')
cursor = conn.cursor()

# 获取2011-1-1到2017-7-31的所有交易日列表
cursor.execute("select distinct date from market where date between '2011-1-1' and '2017-7-31' order by date asc")
days = [x[0] for x in cursor.fetchall()]

# 获取2011-1-1到2017-7-31三大交易所的成交额列表，成交额处理为千万做单位，四舍五入到个位
sql = "select sum(turnover) from market where market='%s' and date between '2011-1-1' and '2017-7-31' group by date order by date asc"
cursor.execute(sql % 'SHFE')
shfe = [round(x[0]/1000) for x in cursor.fetchall()]
cursor.execute(sql % 'DCE')
dce = [round(x[0]/1000) for x in cursor.fetchall()]
cursor.execute(sql % 'CZCE')
czce = [round(x[0]/1000) for x in cursor.fetchall()]

1598L

1598L

1598L

1598L

In [3]:
from pyecharts import Line
line = Line("三大交易所成交额折线图/千万元", width=1000)
attr = [x.strftime('%Y-%m-%d') for x in days]
line.add("上海", attr, shfe, is_datazoom_show=True, tooltip_tragger='axis')
line.add("大连", attr, dce, is_datazoom_show=True, tooltip_tragger='axis')
line.add("郑州", attr, czce, is_datazoom_show=True, tooltip_tragger='axis')
line

In [4]:
shfe_ma30 = MA(shfe)
dce_ma30 = MA(dce)
czce_ma30 = MA(czce)

In [5]:
line_ma30 = Line("三大交易所成交额MA30折线图/千万元", width=1000)
attr = [x.strftime('%Y-%m-%d') for x in days]
line_ma30.add("上海", attr, [None]*29+shfe_ma30, is_datazoom_show=True, tooltip_tragger='axis')
line_ma30.add("大连", attr, [None]*29+dce_ma30, is_datazoom_show=True, tooltip_tragger='axis')
line_ma30.add("郑州", attr, [None]*29+czce_ma30, is_datazoom_show=True, tooltip_tragger='axis')
line_ma30

In [6]:
from pandas import Series
s = Series(data=[corr(shfe_ma30, dce_ma30), corr(shfe_ma30, czce_ma30), corr(dce_ma30, czce_ma30)], 
           index=['corr(上海，大连)', 'corr(上海，郑州)', 'corr(大连，郑州)'])
s

corr(上海，大连)    0.713180
corr(上海，郑州)    0.001011
corr(大连，郑州)    0.019306
dtype: float64

In [7]:
# 柱状统计图统计交易额的分布
width = [700,600,250]    # 上海、大连、郑州
begin = [16500, 6300, 4750]    # 同上
bins = [int((max(shfe)-begin[0])/width[0])+1, int((max(dce)-begin[1])/width[1])+1, int((max(czce)-begin[2])/width[2])+1]     # 同上

hist_shfe = [0]*bins[0]
for x in shfe:
    x -= begin[0]
    hist_shfe[int(x/width[0])] += 1
    
hist_dce = [0]*bins[1]
for x in dce:
    x -= begin[1]
    hist_dce[int(x/width[1])] += 1
    
hist_czce = [0]*bins[2]
for x in czce:
    x -= begin[2]
    hist_czce[int(x/width[2])] += 1
    
bar_shfe = Bar('上海交易额分布图', width=1000)
bar_shfe.add('上海', ['%d~%d' % (begin[0]+i*width[0],begin[0]+(i+1)*width[0]) for i in range(bins[0])], hist_shfe, is_datazoom_show=True)

bar_dce = Bar('大连交易额分布图', width=1000)
bar_dce.add('大连', ['%d~%d' % (begin[1]+i*width[1],begin[1]+(i+1)*width[1]) for i in range(bins[1])], hist_dce, is_datazoom_show=True)

bar_czce = Bar('郑州交易额分布图', width=1000)
bar_czce.add('郑州', ['%d~%d' % (begin[2]+i*width[2],begin[2]+(i+1)*width[2]) for i in range(bins[2])], hist_czce, is_datazoom_show=True)

bar_shfe
bar_dce
bar_czce

In [8]:
# 三大交易所收益率分布
from histogram import Histogram
hist_shfe = Histogram([math.log(shfe[i+1])-math.log(shfe[i]) for i in range(len(shfe)-1)], 100, 0.02)
hist_dce = Histogram([math.log(dce[i+1])-math.log(dce[i]) for i in range(len(dce)-1)], 100, 0.02)
hist_czce = Histogram([math.log(czce[i+1])-math.log(czce[i]) for i in range(len(czce)-1)], 100, 0.02)
hist_shfe.get_echarts("上海")
hist_dce.get_echarts("大连")
hist_czce.get_echarts("郑州")